In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ppscore as pps
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
import numpy as np
from sklearn import metrics
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    confusion_matrix,
    classification_report,
    accuracy_score,
)

# Employee turnover: Predecir la probabilidad de que un empleado renuncie usando Machine Learning
### ¿Cómo podemos retener nuestro talento?

Es una pregunta que toda organización del mundo debería hacerse.
El talento es el insumo de mayor valor para cada organización,
dado que es el mayor generador de valor:
No importan los recursos si no hay gente habilidosa, motivada detrás de ellos.

Para poder retener el talento, es necesario entender
la razón de la salida de los empleados: si se sabe quién está en riesgo
 de abandonar la organización, se pueden tomar medidas preventivas.

En el presente Jupyter notebook, se ilustrará el proceso
para la obtención de un modelo preliminar de
Machine Learning para la predicción de la probabilidad de que
un empleado abandone una organización. Para esto,
se utilizará un dataset de Kaggle que contiene una serie
 de features asociadas a los empleados, además de
 una variable binaria que determina si el empleado abandonó o no la organización.

A grandes rasgos, los pasos que se seguirán serán:

* **Carga y validación de los datos**:
 Se verifica que los datos a usar sean integrales, que no haya faltantes, que sus valores sean coherentes con la realidad.
* **Análisis Exploratorio**:
Se exploran los datos para detectar posibles interacciones de interés. En el caso de dataset con muchos features, se usaría como base para empezar con los modelos.
* **Preparación de los datos**:
 Se realiza preparación y limpieza de datos para los modelos (Normalizar, Estandarizar, Upsampling/Downsampling, generación de datos sintéticos, rellenado de faltantes, entre otras posibilidades). Adicionalmente, se separa un fragmento de los datos para posteriormente probar los modelos en ellos.
* **Modelado inicial**:
Se prueba con algunos modelos básicos.
* **Optimización de hiper parámetros**:
Se aproxima a la mejor combinación de hiper parámetros del modelo seleccionado para los datos que se tienen.
* **Selección del mejor modelo**:
 Con los datos separados en el punto 3, se prueban todos los modelos y se elige aquel con un mejor desempeño.


## Carga y exploración básica

Los features del dataset son:

- **satisfaction_level**: Nivel de Satisfacción, de 0 a 1 (Flotante).
- **last_evaluation**: Años desde la última evaluación de desempeño (Flotante)
- **number_project**: Cantidad de proyectos terminados durante la vinculación laboral (Entera)
- **average_montly_hours**: Horas promedio mensuales trabajadas (Entera)
- **time_spend_company**: Años de vinculación en la compañía (Entera)
- **Work_accident**: Si el empleado tuvo o no Accidente de trabajo (Binaria)
- **left**: Si el empleado dejo o no el trabajo (Variable Respuesta) (Binaria)
- **promotion_last_5years**: Si el empleado tuvo o no un ascenso en los últimos 5 años (Binaria).
- **sales**: Departamento al que el empleado estuvo vinculado (Categórica).
- **salary**: Nivel relativo del salario (low, medium, high) (Categórica)

Se cargan los datos y se validan ciertos elementos, a saber:

* Que no haya datos faltantes
* Que todos los datos sean coherentes con las variables
* Que la variable respuesta esté balanceada

Después de estas validaciones iniciales, se modifican los nombres de las variables
para su interpretabilidad y se realiza un Análisis de Datos Exploratorio para conocer
mejor el dataset.

In [ ]:
df = pd.read_csv("datos.csv")
df

In [ ]:
# Verificación de datos nulos en el dataset
df.isnull().sum()

In [ ]:
# Para variables categoricas y binarias, verificación de que no hayan elementos mal escritos o similar.
# Para esto, se imprimen los valores únicos de cada variable
variables_bin_cat = [
    "Work_accident",
    "left",
    "promotion_last_5years",
    "sales",
    "salary",
]
[(i, df[i].unique(), len(df[i].unique())) for i in variables_bin_cat]

In [ ]:
pd.get_dummies(df["sales"])

In [ ]:
df

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
x = pd.get_dummies(df.drop("salary", axis=1))
y = df["salary"]

In [ ]:
modelo = RandomForestClassifier()
modelo.fit(x, y)

In [ ]:
modelo.predict(x)

In [ ]:
sum(modelo.predict(x) == y) / len(y)

In [ ]:
# Para la verificación de variables númericas continuas, se usa DataFrame.describe() para validar el rango de los datos
df.describe()

In [ ]:
# Se renombra "sales" como "department", "salary" como "salary_level", "left" como "employee_left"
# Esto es para dar nombres más interpretables de las variables
df = df.rename(
    columns={
        "sales": "department",
        "salary": "salary_level",
        "left": "employee_left",
        "average_montly_hours": "average_monthly_hours",
    }
)
# Se convierten las variables a tipo Categoría para facilitar el uso de modelos
df = df.astype({"department": "category", "salary_level": "category"})
df.dtypes

In [ ]:
# Se verifica si la variable respuesta se encuentra balanceada en el dataset, mostrando el conteo de la misma
df.groupby("employee_left")["department"].count().plot(kind="bar")
plt.show()

In [ ]:
# Se realiza un mapa de calor de Predictive Power Score (PPS).
# El PPS es una métrica que mide el poder predictivo de una variable sobre otra.
# Esta se utiliza para observar correlaciones y posibles variables de interés para los modelos
matrix_df = pps.matrix(df)[["x", "y", "ppscore"]].pivot(
    columns="x", index="y", values="ppscore"
)
plt.figure(figsize=(15, 5))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)
plt.show()

In [ ]:
matrix_df

In [ ]:
def get_X_y(df, y_name):
    """ """
    y = [y_name]
    X = [col for col in df.columns if col not in y]
    y = df[y].copy().values.flatten()
    X = pd.get_dummies(df[X].copy())
    return X, y


def data_preprocessing_up_or_down_sample(X, y, sample="up", test_size=0.2):
    """ """

    # Use the sample parameter to define local variables to select the correct
    # method
    a, b = 0, 0
    if sample == "up":
        a, b = 1, 0
    if sample == "down":
        a, b = 0, 1

    # Apply the normal train_test_split to the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    # Using the a and b local variables, apply downsampling or upsampling only
    # if the sample parameter is "up" or "down".

    if a + b >= 1:
        X_train_temp, y_train_temp = resample(
            X_train[y_train == a],
            y_train[y_train == a],
            n_samples=X_train[y_train == b].shape[0],
        )
        X_train = np.concatenate((X_train[y_train == b], X_train_temp))
        y_train = np.concatenate((y_train[y_train == b], y_train_temp))
    return (X_train, X_test, y_train, y_test)


def plot_roc_conf_matrix(y_test, X_test, model, model_name):
    """ """
    try:
        y_pred = model.predict_classes(X_test)
    except:
        y_pred = model.predict(X_test)
    cm = metrics.confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt="g", cmap="Blues")
    plt.title(model_name + " - Matriz de confusión", y=1.1, fontdict={"fontsize": 21})
    plt.xlabel("Predicted", fontdict={"fontsize": 14})
    plt.ylabel("Actual", fontdict={"fontsize": 14})

    print(classification_report(y_test, y_pred))
    plt.subplot(1, 2, 2)

    rocauc_plot(model, model_name, y_test, X_test)


def rocauc_plot(model, model_name, y_test, X_test):
    """ """
    try:
        auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
    except:
        auc = roc_auc_score(y_test, model.predict(X_test))
        fpr, tpr, thresholds = roc_curve(y_test, model.predict(X_test))
    plt.plot(fpr, tpr, label=model_name + " AUC = {:.5f}".format(auc))
    plt.title("Curva(s) ROC", fontdict={"fontsize": 21})
    plt.xlabel("False positive rate", fontdict={"fontsize": 13})
    plt.ylabel("True positive rate", fontdict={"fontsize": 13})
    plt.legend(loc="lower right")
    plt.plot([0, 1], [0, 1], "r--")

In [ ]:
df[["employee_left", "number_project", "satisfaction_level", "salary_level"]]

In [ ]:
X, y = get_X_y(
    df[
        ["employee_left", "average_monthly_hours", "satisfaction_level", "salary_level"]
    ],
    "employee_left",
)
X_train, X_test, y_train, y_test = data_preprocessing_up_or_down_sample(
    X, y, "up", test_size=0.2
)
X_train = pd.DataFrame(columns=X.columns, data=X_train)
for columna in [
    "average_monthly_hours",
    "salary_level_high",
    "salary_level_low",
    "salary_level_medium",
]:
    X_train[columna] = X_train[columna].astype(int)

param_grid = {
    "randomforestclassifier__min_samples_leaf": np.arange(1, 11, 2),
    "randomforestclassifier__n_estimators": np.arange(100, 1000 + 100, 250),
    "randomforestclassifier__criterion": ["gini", "entropy"],
}
pipe = make_pipeline(StandardScaler(), RandomForestClassifier(class_weight="balanced"))
clf = GridSearchCV(
    pipe, param_grid=param_grid, cv=5, refit=True, scoring="f1", n_jobs=-1
)
clf.fit(X_train, y_train)

In [ ]:
X_train.columns

In [ ]:
plot_roc_conf_matrix(y_test, X_test, clf, "Random Forest")
plt.show()

In [ ]:
import joblib

# SE GUARDA EL MODELO
joblib.dump(clf, "model ex.pkl")

In [ ]:
X_train.head(1)

# Clase 1: Pydantic

In [ ]:
from pydantic import BaseModel as BM
from pydantic import Field
from typing import Literal
from pydantic import ValidationError

In [ ]:
class InputModeloNo:
    def __init__(self, salary_level: str):
        if isinstance(salary_level, str):
            if salary_level in ["low", "medium", "high"]:
                self.salary_level = salary_level
                print(":)")
            else:
                print(":'(")
        else:
            print(":'(")

In [ ]:
InputModeloNo(salary_level="low")

In [ ]:
class InputModelo(BM):
    """
    Clase que define las entradas del modelo según las verá el usuario.
    """

    average_monthly_hours: int = Field(
        ge=96, le=310, description="Horas promedio mensuales trabajadas"
    )
    satisfaction_level: float = Field(ge=0, le=1)
    salary_level: Literal["high", "low", "medium"]

    class Config:
        scheme_extra = {
            "example": {
                "average_monthly_hours": 201,
                "satisfaction_level": 0.42,
                "salay_level": "high",
            }
        }


class OutputModelo(BM):
    """
    Clase que define la salida del modelo según la verá el usuario.
    """

    employee_left: float = Field(ge=0, le=1)

    class Config:
        scheme_extra = {
            "example": {
                "employee_left": 0.69,
            }
        }

In [ ]:
import joblib

clf = joblib.load("model ex.pkl")

In [ ]:
average_monthly_hours = 201
satisfaction_level = 0
salary_level = "high"

In [ ]:
InputModelo(
    average_monthly_hours=average_monthly_hours,
    satisfaction_level=satisfaction_level,
    salary_level=salary_level,
)

In [ ]:
InputModelo(
    average_monthly_hours=average_monthly_hours,
    satisfaction_level=satisfaction_level,
    salary_level=salary_level,
)

salary_levels = [0] * 3

# Crea el DataFrame en el mismo orden las columnas del X_train

data_predict = pd.DataFrame(
    columns=[
        "average_monthly_hours",
        "satisfaction_level",
        "salary_level_high",
        "salary_level_low",
        "salary_level_medium",
    ],
    data=[[average_monthly_hours, satisfaction_level, *salary_levels]],
)

# Pone el 1 en la columna que debe ir el 1

data_predict[
    [
        x
        for x in data_predict.columns
        if ((salary_level in x) and (x.startswith("salary_level_")))
    ]
] = 1


pd.DataFrame(clf.predict_proba(data_predict)[:, 1]).rename(columns={0: "prediccion"})

In [ ]:
pd.DataFrame(clf.predict_proba(data_predict)[:, 1]).rename(columns={0: "prediccion"})

# Clase 2: FastAPI

In [ ]:
# Se carga el modelo

import joblib

first_model = joblib.load("model ex.pkl")

### Creamos un archivo llamado classes.py para guardar todo lo que hacemos con las clases

De la sesión pasada, tenmos parte del archivo:

In [ ]:
from pydantic import BaseModel as BM
from pydantic import Field
from typing import Literal
import pandas as pd
import joblib


class InputModelo(BM):
    """
    Clase que define las entradas del modelo según las verá el usuario.
    """

    average_monthly_hours: int = Field(
        ge=96, le=310, description="Horas promedio mensuales trabajadas"
    )
    satisfaction_level: float = Field(ge=0, le=1)
    salary_level: Literal["high", "low", "medium"]

    class Config:
        scheme_extra = {
            "example": {
                "average_monthly_hours": 201,
                "satisfaction_level": 0.42,
                "salay_level": "high",
            }
        }


class OutputModelo(BM):
    """
    Clase que define la salida del modelo según la verá el usuario.
    """

    employee_left: float = Field(ge=0, le=1)

    class Config:
        scheme_extra = {
            "example": {
                "employee_left": 0.69,
            }
        }


class APIModelBackEnd:
    def __init__(self, average_monthly_hours, satisfaction_level, salary_level):

        self.average_monthly_hours = average_monthly_hours
        self.satisfaction_level = satisfaction_level
        self.salary_level = salary_level

    def cargar_modelo(self, nombre_modelo: str = "model ex.pkl"):
        self.model = joblib.load(nombre_modelo)

    def preparar_datos_modelo(self):

        average_monthly_hours = self.average_monthly_hours
        satisfaction_level = self.satisfaction_level
        salary_level = self.salary_level

        salary_levels = [0] * 3

        # Crea el DataFrame en el mismo orden las columnas del X_train

        data_predict = pd.DataFrame(
            columns=[
                "average_monthly_hours",
                "satisfaction_level",
                "salary_level_high",
                "salary_level_low",
                "salary_level_medium",
            ],
            data=[[average_monthly_hours, satisfaction_level, *salary_levels]],
        )

        # Pone el 1 en la columna que debe ir el 1

        data_predict[
            [
                x
                for x in data_predict.columns
                if ((salary_level in x) and (x.startswith("salary_level_")))
            ]
        ] = 1

        return data_predict

    def predecir(self):
        self.cargar_modelo()
        x = self.preparar_datos_modelo()
        prediccion = pd.DataFrame(self.model.predict_proba(x)[:, 1]).rename(
            columns={0: "employee_left"}
        )

        return prediccion.to_dict(orient="records")

In [ ]:
first_class = APIModelBackEnd(
    average_monthly_hours=average_monthly_hours,
    satisfaction_level=satisfaction_level,
    salary_level=salary_level,
)
first_class.predecir()[0]

In [ ]:
from fastapi import FastAPI
from typing import List

app = FastAPI(title="Mi  API de ML", version="1.0.1")


@app.post("/predict", response_model=List[OutputModelo])
def predecir_probabilidad(inputs: List[InputModelo]):
    respuestas = list()
    for Input in inputs:
        first_model = APIModelBackEnd(
            Input.average_monthly_hours, Input.satisfaction_level, Input.salary_level
        )

        respuestas.append(first_model.predecir()[0])

    return respuestas

In [ ]:
average_montly_hours = 201
satisfaction_level = 0
salary_level = "low"
test = APIModelBackEnd(
    average_montly_hours=average_montly_hours,
    satisfaction_level=satisfaction_level,
    salary_level=salary_level,
)

In [ ]:
OutputModelo(employee_left=test.predecir()[0]["employee_left"])

In [ ]:
import requests

average_monthly_hours = 300
satisfaction_level = 0.12
salary_level = "medium"

request_data = [
    {
        "average_monthly_hours": average_monthly_hours,
        "satisfaction_level": satisfaction_level,
        "salary_level": salary_level,
    }
]

data_cleaned = str(request_data).replace("'", '"')

url_api = "https://api-diplomadopython.herokuapp.com/predict"

pred = requests.post(url=url_api, data=data_cleaned).text

pred

In [ ]:
import sklearn

sklearn.__version__

# Clase 3

In [ ]:
# leemos los datos de la fuente

df = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
)
df

In [ ]:
df_col = df[df["Country/Region"].apply(lambda x: x in ["Colombia"])].copy()
df_col

In [ ]:
melted = df_col.melt(
    var_name="Fechas",
    value_name="confirmados",
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
)
melted

In [ ]:
import datetime

melted["Fechas"] = melted["Fechas"].apply(
    lambda x: datetime.datetime.strptime(x, "%m/%d/%y")
)

In [ ]:
melted

In [ ]:
import plotly.express as px

In [ ]:
def dibujar_serie_tiempo(
    data, x="Fechas", y="confirmados", title="Casos de COVID Confirmados en Colombia"
):
    fig = px.line(
        data,
        x=x,
        y=y,
        title=title,
        color_discrete_sequence=["red", "blue"],
    )

    fig.update_layout(yaxis_title="Casos confirmados", xaxis_title="Fecha")
    # Esto elimina el color del fondo:
    fig.update_layout(
        {"plot_bgcolor": "rgba(0,0,0,0)", "paper_bgcolor": "rgba(0,0,0,0)"}
    )

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list(
                [
                    dict(step="day", stepmode="backward", label="1 semana", count=7),
                    dict(step="month", stepmode="backward", label="1 mes", count=1),
                    dict(step="month", stepmode="backward", label="3 meses", count=3),
                    dict(step="month", stepmode="backward", label="6 meses", count=6),
                    dict(label="Todos", step="all"),
                ]
            )
        ),
    )
    fig.update_traces(
        hovertemplate="<b><i>"
        + "Casos confirmados"
        + "</i></b>: %{y} <br><b><i>"
        + "Fecha"
        + "</i></b>: %{x} <extra></extra>"
    )
    return fig


# fig.update_xaxes(dtick='M1', ticklabelmode='period', tickformat="%d %b\n%Y")

fig = dibujar_serie_tiempo(melted)
fig.show()

In [ ]:
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list(
            [
                dict(step="day", stepmode="backward", label="1 semana", count=7),
                dict(step="month", stepmode="backward", label="1 mes", count=1),
                dict(step="month", stepmode="backward", label="3 meses", count=3),
                dict(step="month", stepmode="backward", label="6 meses", count=6),
                dict(label="Todos", step="all"),
            ]
        )
    ),
)

In [ ]:
def plot_heatmap(df: pd.DataFrame, x: str, y: str):
    data_heatmap = (
        df.reset_index()[[x, y, "index"]]
        .groupby([x, y])
        .count()
        .reset_index()
        .pivot(x, y, "index")
        .fillna(0)
    )
    fig = px.imshow(
        data_heatmap,
        color_continuous_scale="Reds",
        aspect="auto",
        title=f"Heatmap {x} vs {y}",
    )
    fig.update_traces(
        hovertemplate="<b><i>"
        + y
        + "</i></b>: %{y} <br><b><i>"
        + x
        + "</i></b>: %{x} <br><b><i>Conteo interacción variables</i></b>: %{z}<extra></extra>"
    )
    return fig

In [ ]:
df2 = pd.read_csv("datos.csv")
df2

In [ ]:
plot_heatmap(df2, x="sales", y="salary")

# Clase 4

In [ ]:
plotly.__version__

In [ ]:
int("5")

In [ ]:
isinstance("5", str)

In [ ]:
a = set([1])
b = set([1, 2, 3, 4, 5])

In [ ]:
%%timeit
any(a.intersection(b))

In [ ]:
%%timeit
1 in [1, 2, 3, 4, 5]

In [ ]:
5 == "5"

In [ ]:
5 == 5.0

In [ ]:
for i in "hola":
    print(i)

In [ ]:
for i, j in enumerate([3, 2, 1, 0]):
    print(i, j)

In [ ]:
for i, j in enumerate("holi"):
    print(i, j)

In [ ]:
for i in {0: 1, 1: 1, 2: 1}:
    print(i)

In [ ]:
def funcion(holi: str = "d"):
    t = 1 == 2 | 9 == 9 | 1 == 3
    return t

In [ ]:
def ejemplo(txt:str):
    return txt + 'Hola'

In [ ]:
def test_ejemplo(txt:str):
    assert "holaHola"==ejemplo(txt)

In [ ]:
test_ejemplo("hola")

In [ ]:
import re

In [ ]:
pd.DataFrame(columns=['Hola'], data= [['aihdaio'], ['92487932ada']])['Hola'].apply(lambda x: str(x).upper()).apply(lambda x: re.sub("[9]", "holi",x))

In [ ]:
pd.DataFrame(columns=["Hola"], data=[["aihdaio"], ["92487932ada"]])["Hola"].apply(
    lambda x: str(x).upper()
).apply(lambda x: re.sub("[9]", "holi", x))

In [ ]:
pd.DataFrame(columns=["Hola"], data=[["aihdaio"], ["92487932ada"]])["Hola"].apply(
    lambda x: str(x).upper()
).apply(
    lambda x: re.sub("[9]", "holi", x)
)

In [ ]:
(
    pd.DataFrame(columns=["Hola"], data=[["aihdaio"], ["92487932ada"]])["Hola"]
    .apply(lambda x: str(x).upper())
    .apply(lambda x: re.sub("[9]", "holi", x))
)

# Clase 5

In [1]:
from decouple import config, UndefinedValueError

In [ ]:
# Deben crear un archivo '.env' con:

'''
EJEMPLO = "Acá en comillas ponen el valor que quieren darle a la variable"
'''
config("EJEMPLO")

In [8]:
def login(usuario, contraseña):
    """
    ESTO ES UN EJEMPLO DE LO QUE QUEREMOS EVITAR QUE QUEDE USANDO VARIABLES DE ENTORNO

    NO USAR
    NO COPIAR Y PEGAR


    NO USAR
    """

    # ESTO ES UN PECADO CAPITAL
    if (usuario == "user") and (contraseña == "password"):
        print("sesion iniciada")
    else:
        print("datos invalidos")


login("user", "password")

sesion iniciada


In [3]:
try:
    ejemplo = config("EJEMPLO")
    
except UndefinedValueError:
    ejemplo = 'ERROR'


    

In [5]:
try:
    import pandas as pd
except ModuleNotFoundError:
    !pip install pandas